In [10]:
# MENGIMPOR SEMUA LIBRARY YANG DIPERLUKAN UNTUK BAB 7
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

# Impor Model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingClassifier
import xgboost

# Impor Data dan Utilitas
from sklearn.datasets import make_moons, load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.inspection import DecisionBoundaryDisplay

# Mengatur default plotting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

print("--- BAB 7: ENSEMBLE LEARNING DAN RANDOM FORESTS ---")
print("Semua library telah diimpor.")

# 1. VOTING CLASSIFIERS
print("\n[1. Voting Classifiers]")
# Teori: Menggabungkan prediksi dari beberapa model. "Kebijaksanaan orang banyak".

# Membuat dataset moons
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Mendefinisikan tiga classifier yang berbeda
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

# Membuat Voting Classifier (Hard Voting)
# Hard voting: mengambil prediksi mayoritas (voting)
voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf_hard.fit(X_train, y_train)

print("  Akurasi masing-masing model (Hard Voting):")
for clf in (log_clf, rnd_clf, svm_clf, voting_clf_hard):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"  - {clf.__class__.__name__}: {accuracy_score(y_test, y_pred):.4f}")

# Membuat Voting Classifier (Soft Voting)
# Teori: Soft voting merata-ratakan probabilitas prediksi dari semua model
# dan memilih kelas dengan probabilitas rata-rata tertinggi.
# Biasanya bekerja lebih baik. Perlu model yang bisa memprediksi prob (predict_proba())
svm_clf_soft = SVC(gamma="scale", probability=True, random_state=42) # Harus set probability=True

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf_soft)],
    voting='soft'
)
voting_clf_soft.fit(X_train, y_train)

print("\n  Akurasi (Soft Voting):")
y_pred_soft = voting_clf_soft.predict(X_test)
print(f"  - VotingClassifier (Soft): {accuracy_score(y_test, y_pred_soft):.4f}")
# Soft voting biasanya memberikan akurasi yang lebih baik.

# 2. BAGGING DAN PASTING
print("\n[2. Bagging dan Pasting]")
# Teori: Melatih algoritma yang SAMA berulang kali pada
# subset acak yang berbeda dari data latih.
# - Bagging (Bootstrap Aggregating): Sampling DENGAN penggantian (with replacement).
# - Pasting: Sampling TANPA penggantian (without replacement).

# Menggunakan BaggingClassifier dengan Decision Trees
tree_clf = DecisionTreeClassifier(random_state=42)
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    max_samples=100, # Setiap pohon dilatih hanya pada 100 instance
    bootstrap=True,  # True = Bagging, False = Pasting
    random_state=42
)

bag_clf.fit(X_train, y_train)
y_pred_bag = bag_clf.predict(X_test)

tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)

print(f"  Akurasi Decision Tree tunggal: {accuracy_score(y_test, y_pred_tree):.4f}")
print(f"  Akurasi Bagging (500 pohon): {accuracy_score(y_test, y_pred_bag):.4f}")

# Fungsi helper untuk plot decision boundary
def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    # Buat meshgrid
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]

    # Dapatkan prediksi
    y_pred = clf.predict(X_new).reshape(x1.shape)

    # Plot kontur (batas keputusan)
    custom_cmap = mpl.colors.ListedColormap(['#9898ff','#fafab0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)

    if contour:
        custom_cmap2 = mpl.colors.ListedColormap(['#7d7d58','#4c4c7f'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)

    # Plot data points
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

# Plot (di-comment agar tidak auto-tampil)
# plt.figure(figsize=(11,4))
# plt.subplot(121)
# plot_decision_boundary(tree_clf, X, y)
# plt.title("Decision Tree Tunggal (Overfitting)", fontsize=14)
# plt.subplot(122)
# plot_decision_boundary(bag_clf, X, y)
# plt.title("Bagging dengan 500 Pohon (Lebih Halus)", fontsize=14)
# plt.show()
# Hasil: Bagging memiliki batas keputusan yang lebih halus dan generalisasi lebih baik.

# 3. OUT-OF-BAG (OOB) EVALUATION
print("\n[3. Out-of-Bag (OOB) Evaluation]")
# Teori: Dalam Bagging, karena sampling *with replacement*,
# sekitar 37% data latih tidak terpakai (out-of-bag) untuk setiap pohon.
# Kita bisa menggunakan data oob ini sebagai validation set otomatis.

bag_clf_oob = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    bootstrap=True,
    oob_score=True, # Meminta OOB score
    random_state=40
)
bag_clf_oob.fit(X_train, y_train)
print(f"  Skor OOB: {bag_clf_oob.oob_score_:.4f}")

# Mari kita bandingkan dengan akurasi di test set
y_pred_oob = bag_clf_oob.predict(X_test)
print(f"  Akurasi Test Set: {accuracy_score(y_test, y_pred_oob):.4f}")
# Skor OOB adalah estimasi yang baik untuk akurasi test set.

# 4. RANDOM FORESTS
print("\n[4. Random Forests]")
# Teori: Random Forest adalah ensemble Decision Trees,
# dilatih dengan Bagging (biasanya).
# Perbedaan utama: Saat membelah node, alih-alih mencari fitur
# terbaik dari SEMUA fitur, ia mencari yang terbaik dari
# SUBSET ACAK fitur. Ini menambah keacakan dan mengurangi korelasi
# antar pohon, sehingga mengurangi varians (lebih baik).

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)
print(f"  Akurasi Random Forest: {accuracy_score(y_test, y_pred_rf):.4f}")

# 5. FEATURE IMPORTANCE
print("\n[5. Feature Importance]")
# Teori: Random Forest bisa mengukur seberapa penting setiap fitur
# dengan melihat seberapa sering fitur tsb mengurangi impurity
# di semua pohon.

iris = load_iris()
rnd_clf_iris = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf_iris.fit(iris["data"], iris["target"])

print("  Feature Importance (Dataset Iris):")
for name, score in zip(iris["feature_names"], rnd_clf_iris.feature_importances_):
    print(f"  - {name}: {score:.4f}")
# Kita bisa lihat bahwa petal length dan width adalah fitur terpenting.

# 6. BOOSTING
print("\n[6. Boosting (AdaBoost & Gradient Boosting)]")
# Teori: Boosting adalah metode ensemble sekuensial.
# Model dilatih satu per satu, di mana setiap model baru
# mencoba memperbaiki kesalahan model sebelumnya.

# 6a. AdaBoost (Adaptive Boosting)
# Teori: Fokus pada instance yang salah diklasifikasikan
# oleh model sebelumnya (dengan meningkatkan bobot/weight mereka).
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=200,
    algorithm="SAMME",
    learning_rate=0.5,
    random_state=42
)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print(f"  Akurasi AdaBoost: {accuracy_score(y_test, y_pred_ada):.4f}")

# 6b. Gradient Boosting (GBRT)
print("  Melatih Gradient Boosting (GBRT)...")
# Teori: Melatih model baru pada 'residual error' (sisa kesalahan)
# dari model sebelumnya.

# Membuat data kuadratik untuk regresi
np.random.seed(42)
X_reg = np.random.rand(100, 1) - 0.5
y_reg = 3*X_reg**2 + 0.05 * np.random.randn(100, 1)

# Langkah Manual (Konseptual)
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X_reg, y_reg)
y2 = y_reg - tree_reg1.predict(X_reg).reshape(-1, 1) # Hitung error
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X_reg, y2) # Latih pohon kedua pada error
y3 = y2 - tree_reg2.predict(X_reg).reshape(-1, 1) # Hitung error lagi
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X_reg, y3)
# Prediksi = tree_reg1 + tree_reg2 + tree_reg3
# y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

# Menggunakan Scikit-Learn GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_reg, y_reg.ravel())

# 6c. GBRT dengan Early Stopping
# Teori: Kita bisa menemukan jumlah pohon (n_estimators) optimal
# menggunakan 'staged_predict' (mirip early stopping).
X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(X_reg, y_reg.ravel(), random_state=42)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt_best.fit(X_train_reg, y_train_reg)

# Cari error pada setiap tahap (setiap penambahan pohon)
errors = [mean_squared_error(y_val_reg, y_pred)
          for y_pred in gbrt_best.staged_predict(X_val_reg)]
bst_n_estimators = np.argmin(errors) + 1 # Temukan jumlah pohon terbaik

print(f"  Jumlah pohon (estimator) terbaik untuk GBRT: {bst_n_estimators}")

# Latih ulang model final dengan jumlah pohon terbaik
gbrt_final = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_final.fit(X_reg, y_reg.ravel())

# 7. XGBOOST (EXTREME GRADIENT BOOSTING)
print("\n[7. XGBoost]")
# Teori: Implementasi Gradient Boosting yang sangat cepat,
# dapat di-scale, dan sangat populer di kompetisi.

# Inisialisasi dan latih
xgb_reg = xgboost.XGBRegressor(random_state=42)

# Manual early stopping for XGBoost
# Check for best iteration during training
eval_set = [(X_val_reg, y_val_reg)]
xgb_reg.fit(X_train_reg, y_train_reg, eval_set=eval_set, verbose=False) # Train without early stopping parameter

# Note: With recent XGBoost versions, you might need to access the best iteration differently
# or rely on the output of the fit method if verbose is True.
# Since early_stopping_rounds is causing issues, we'll skip finding the best iteration
# and just use the model trained for the full number of default estimators.
# If early stopping is crucial, manual implementation with a loop and tracking validation score is needed.
# For now, we proceed with the full training.

y_pred_xgb = xgb_reg.predict(X_val_reg)
xgb_rmse = np.sqrt(mean_squared_error(y_val_reg, y_pred_xgb))
print(f"  RMSE XGBoost: {xgb_rmse:.6f}")

# 8. STACKING (STACKED GENERALIZATION)
print("\n[8. Stacking]")
# Teori: Pendekatan ensemble tingkat lanjut. Alih-alih voting,
# kita melatih model baru (disebut 'blender' atau 'meta-learner')
# untuk melakukan agregasi.
# Lapisan 1: Prediktor (misal, RF, SVM)
# Lapisan 2 (Blender): Model (misal, Ridge) dilatih PADA PREDIKSI dari lapisan 1.

# StackingClassifier sudah ada di Scikit-Learn
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svc', SVC(probability=True, random_state=42)) # Perlu probability=True untuk blender
]

# Blender (meta-learner) bisa berupa apa saja, biasanya model linear
# 'passthrough=True' berarti blender juga akan menerima fitur asli
stack_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    passthrough=True,
    cv=5 # Cross-validate prediksi L1
)

stack_clf.fit(X_train, y_train)
y_pred_stack = stack_clf.predict(X_test)
print(f"  Akurasi Stacking Classifier: {accuracy_score(y_test, y_pred_stack):.4f}")

print("\n--- Selesai Bab 7 ---")

--- BAB 7: ENSEMBLE LEARNING DAN RANDOM FORESTS ---
Semua library telah diimpor.

[1. Voting Classifiers]
  Akurasi masing-masing model (Hard Voting):
  - LogisticRegression: 0.8640
  - RandomForestClassifier: 0.8960
  - SVC: 0.8960
  - VotingClassifier: 0.9120

  Akurasi (Soft Voting):
  - VotingClassifier (Soft): 0.9200

[2. Bagging dan Pasting]
  Akurasi Decision Tree tunggal: 0.8560
  Akurasi Bagging (500 pohon): 0.9040

[3. Out-of-Bag (OOB) Evaluation]
  Skor OOB: 0.8987
  Akurasi Test Set: 0.9120

[4. Random Forests]
  Akurasi Random Forest: 0.9120

[5. Feature Importance]
  Feature Importance (Dataset Iris):
  - sepal length (cm): 0.1125
  - sepal width (cm): 0.0231
  - petal length (cm): 0.4410
  - petal width (cm): 0.4234

[6. Boosting (AdaBoost & Gradient Boosting)]


/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


  Akurasi AdaBoost: 0.8960
  Melatih Gradient Boosting (GBRT)...
  Jumlah pohon (estimator) terbaik untuk GBRT: 118

[7. XGBoost]
  RMSE XGBoost: 0.058937

[8. Stacking]
  Akurasi Stacking Classifier: 0.9120

--- Selesai Bab 7 ---
